In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Data Generation
We generate 5000 training samples of solutions to the 1D Heat equation, with the input being an initial condition and a $\alpha$ distribution,
and the output being the heat distribution after 0.1 seconds.

## Initial Condition
We superimpose randomly generated Gaussian distributions to create the initial heat distribution.

## $\alpha$ Distribution
We create a randomized even-order polynomial which takes on only positive values along the computational domain.

## Final Solution
We use a spectral method to calculate the final solution using 100 timesteps.

In [ ]:
def gaussian(size, center, sigma):
    x = np.linspace(-1, 1, size)
    return np.exp(-(x - center) ** 2 / (2 * sigma ** 2))

In [ ]:
def random_ic(size, num_gaussians):
    rng = np.random.default_rng()

    y = np.zeros(size)
    for _ in range(num_gaussians):
        y += gaussian(size, rng.uniform(-0.5, 0.5), rng.uniform(0.05, 0.1))

    return y * rng.uniform(1, 10) #Scale

In [ ]:
def random_alpha(size):
    rng = np.random.default_rng()

    #Make a random even-order polynomial
    k = rng.integers(1, 4) * 2
    x = np.linspace(-1, 1, size)
    y = np.ones_like(x)
    epsilon = 0.2

    for i in range(1, k):
        if i % 2 == 0:
            y *= x - rng.uniform(-2, -1 - epsilon)
        else:
            y *= x - rng.uniform(1 + epsilon, 2)

    y = np.abs(y)

    return y / np.max(y)

In [ ]:
def solve_heat1d(u0, alpha, L, timesteps=100, final_t = 0.1):
    N = u0.size
    dx = L / N
    kappa = 2 * np.pi * np.fft.fftfreq(N, d=dx)
    u0_hat = np.fft.fft(u0)
    u0_hat_ri = np.concatenate((u0_hat.real, u0_hat.imag))

    t = np.linspace(0, final_t, timesteps)

    def rhs(uhat_ri, _, kappa, a):
        uhat = uhat_ri[:N] + 1j * uhat_ri[N:]
        d_uhat = -a**2 * np.power(kappa, 2) * uhat

        return np.concatenate((d_uhat.real, d_uhat.imag))

    full_solution = odeint(rhs, u0_hat_ri, t, args=(kappa, alpha))
    uf_hat_ri = full_solution[-1]
    uf_hat = uf_hat_ri[:N] + 1j * uf_hat_ri[N:]

    return np.real(np.fft.ifft(uf_hat))

In [ ]:
def generate_data(samples):
    L = 1
    nodes = 100
    x = np.linspace(-L/2, L/2, nodes) #(N,)
    dataset = np.zeros(shape=(samples, 3, nodes))

    for k in range(samples):
        y_ic = random_ic(x.size, 3) #(N,)
        a = random_alpha(x.size) #(N,)
        y_f = solve_heat1d(y_ic, a, L) #(N,)

        entry = np.vstack((y_ic, a, y_f))
        dataset[k] = entry

    return dataset

In [ ]:
dataset = generate_data(10)
np.save("test.npy", dataset)